# Import packages

In [ ]:
## import 
import os
import numpy as np
import time
import matplotlib.pyplot as plt
# from tqdm import tqdm
# import h5py
import atomai
import gpax
from sklearn.model_selection import train_test_split
from atomai.utils import get_coord_grid, extract_patches_and_spectra, extract_subimages
import Pyro5.api
import io
import time
from SciFiReaders import IgorIBWReader
from IPython.display import display, clear_output
import atomai as aoi
import torch

In [ ]:
import os
import numpy as np
import time
import matplotlib.pyplot as plt
from IPython.display import display, clear_output

from scipy.signal import find_peaks
from sklearn.model_selection import train_test_split

import gpax
import atomai as aoi


In [ ]:
from aespm import *
from utils import *
from tools import *
# import aespm.tools as at

# Make the connection and custom functions

In [ ]:
host = ''
username = ''
password = ''

folder = r"C:\Users\Asylum User\Documents\Asylum Research Data\240410"

exp = Experiment(folder=folder, connection=[host, username, password])

## Custom functions for taking measurements

In [ ]:
## Commonly used custom functions

def load_ibw(self, folder="C:\\Users\\Asylum User\\Documents\\AEtesting\\data_exchange"):
    '''
    Read the latest ibw file saved in a given folder.
    '''
    fname = get_files(path=folder, client=self.client)[0]
    return ibw_read(fname, connection=self.connection)

exp.add_func(load_ibw)

# Function to move the probe with the given displacement 
def move_tip(self, distance, v0=None, s=None):
    
    # Enable the stage move --> 5 sec, 8 seconds for safety
    move_tip(r=distance, v0=v0, s=s, connection=self.connection)

exp.add_func(move_tip)

# Function to check the file number in a given folder
def check_files(self):
    return check_file_number(path=self.folder, connection=self.connection)
exp.add_func(check_files)

def load_ibw_parameter(self, data):
    
    scan_angle = data.header['ScanAngle']
    xpixels, ypixels = data.header['PointsLines'],data.header['ScanPoints']
    xsize, ysize = data.header['FastScanSize'],data.header['SlowScanSize']

    xfactor = xsize / xpixels
    yfactor = ysize / ypixels
    
    p = {
        'ScanAngle': scan_angle,
        'xpixels': xpixels,
        'ypixels': ypixels,
        'xsize': xsize,
        'ysize': ysize,
        'xfactor': xfactor,
        'yfactor': yfactor,
        'center': np.array([xsize, size]) / 2,
    }
    
    for key in p:
        self.update_param(key=key, value=p[key])
exp.add_func(load_ibw_parameter)

def convert_coord(self, coord):
    '''
    Convert the coordinate from pixel to distance.
    Apply rotation if needed.
    '''
    x, y = coord
    
    if len(np.shape(coord)) == 1:
        x, y = [x], [y]
    scan_angle = self.param['ScanAngle']
    
    # Convert angle to radians
    theta_rad = np.radians(-scan_angle)
    
    # Create 2D rotation matrix
    rot_matrix = np.array([[np.cos(theta_rad), -np.sin(theta_rad)],
                           [np.sin(theta_rad), np.cos(theta_rad)]])
    
    # Apply the rotation matrix to the coordinates
    center = (np.array([self.param['xpixels'], self.param['ypixels']])-1) // 2
    x_rot, y_rot = np.zeros_like(x), np.zeros_like(y)
    for i in range(len(x)):
        x_rot[i], y_rot[i] = np.dot(rot_matrix, (np.array([x[i], y[i]])-center)) + center
    
    # Convert the pixels to the distance
    xfactor, yfactor = self.param['xfactor'], self.param['yfactor']

    positions = np.zeros([len(x), 2])

    for i in range(len(x)):
        positions[i] = np.array([x_rot[i] * xfactor, y_rot[i] * yfactor])

    # Sort the positions according to x first and y second
    pos_sorted = sorted(positions, key=lambda x: (x[1], x[0]))
    return pos_sorted

exp.add_func(convert_coord)

def measure_spectrum(self, coord, v0, basename, index, retry=100):
    
    x, y = self.convert_coord(coord)
    r = np.array([x, y]) - self.param['center']
    self.move_tip(distance=r, v0=v0, s=self.param['sensitivity'])
    self.execute('IVDoItDART')
    
    time.sleep(5)
    retries = 0
    while retries < retry:
        try:
            
            download_file(connection=connection, file_path=os.path.join(folder, '{}_{:04d}.ibw'.format(basename, index)), local_file_name='spec.ibw')
            d = load_ibw(file='spec.ibw', ss=True)
            return d
        except FileNotFoundError:
            retries += 1
            time.sleep(2)
            
exp.add_func(measure_spectrum)

In [2]:
def scal(obj):
    index = len(obj.bias) // 4
    index_up = [2*index, -1]
    index_down = [index, 3*index]
    return np.abs(np.mean(obj.amp_off[index_up])-np.mean(obj.amp_off[index_down]))

2

## Custom functions for DKL

In [ ]:
def generate_seed(self, num_seed, )

In [ ]:
## do measuremnt on seed points:
y_measured_unnor = []
y_measured_raw = []

spm_control('ChangeName', value='PTO_seed_', connection=connection)

for i in range (seed_points):
    print("Working on {}/{} points".format(i+1, seed_points), end='\r')
    if i % 10 == 0: # Re-tune the probe every 10 measurements at the center of the image
        spm_control('ClearForce', connection=connection)
        spm_control('GoThere', wait=1, connection=connection)
        tune_probe(num=1, center=340e3, width=50e3, connection=connection)
        
    x, y = float(indices_measured[i][0]), float(indices_measured[i][1])
    array = spectrum_calc(x,y, basename='PTO_seed', index=i)# ask richard
    # y_measured_raw.append(array)
    current_y = scal(array)
    y_measured_unnor.append(current_y)
    
    clear_output(wait=True)
    plt.figure(figsize=[4,4])
    plt.plot(array.bias, array.amp_off, '.-')
    plt.xlabel('Sample Bias (V)')
    plt.ylabel('Piezo response (a.u.)')
    
    plt.show()


In [ ]:
def get_files(path, retry=10, sleep_time=6e-3, client=None):
    '''
    Get the file names and sort them in the new to old modification time from a given folder.

    Input:
        path    - String: path to the folder where the SPM data is saved locally
        retry   - Int: number of retries if any of the data file is occupied by SPM controller
        sleep_time - float: wait time between the retries
        client  - SSH client returned by utils.return_connection()
    Output:
        fname   - List: a list of filenames in the order of new to old modification time
    '''
    command = 'dir "{}" /b /o:-d'.format(path)
    retries = 0
    while retries < retry:
        try:
            # Run the command and capture its output
            if client == None:
                result = subprocess.run(command, shell=True, capture_output=True, text=True, check=True)
                file_names = result.stdout.split('\n')
                return [os.path.join(path, file) for file in file_names]
            else:
                stdin, stdout, stderr = client.exec_command(command)
                # Read the output of the command
                file_names = stdout.read().decode('utf-8').splitlines()
                return [os.path.join(path, file) for file in file_names]
            
        except FileNotFoundError:
            print("File not found. Retrying {} times...".format(retries), end='\r')
            retries += 1
            time.sleep(sleep_time)
        except PermissionError:
            print("Permission denied. Retrying {} times...".format(retries), end='\r')
            retries += 1
            time.sleep(sleep_time)
    return 0

def check_file_number(path, wait=1e-1, retry=1e4, client=None):
    '''
    Check if the spectrum/topo measurement is done by monitoring the number of files in the save folder.
    It waits until the number of files has changed to return a True value.
    '''
    num = len(get_files(path, client=client))
    num_new = num
    retries = 0
    # Check the file number in the folder until 16 min is passed
    while num_new == num and retries < retry:
        num_new = len(get_files(path, client=client))
        retries += 1
        time.sleep(wait)
    return True

def check_status(mode='Scan', retry=1800, connection=None):
    '''
    Check the status of the measurements. Will hold the workflow until the measurement is done.
    
    Input:
        mode    - String: 'Scan': scan measurement
                          'FMap': Spectral measurement on a grid
                          'Spec': Single spectral measurement
        retry   - Int: number of total retries before giving up this checking
        connection - SSH connection: return by utils.return_connection()
    Output:
        N/A
    Examples:
        spm_control('DownScan', wait=1.5)
        check_status(mode='Scan')
    '''
    
    if mode == 'Scan':
        key = 'ScanStatus'
    elif mode == 'FMap':
        key = 'FMapStatus':
    elif mode == 'Spec':
        key = 
    else:
        print('Mode not supported yet!')
        
    retries = 0
    while retries < retry:
        status = read_spm(key=key, connection=connection)
        if status == 0:
            return True
        elif status == 1:
            retries += 1
        else:
            print('Error in reading the status of the measurement.')

def ibw_read(fname, retry=10, wait=0.1, copy=True, connection=None):
    '''
    Make a copy of realtime saved image and read the wave data from it.
    Four data channels will be read with the order of ZSenor/Height -> Amplitude -> Phase -> Height/Deflection.
    Each channel contains both the trace and retrace scan lines.
    '''
    retries = 0
    if connection==None:
        if copy==True:
            while retries < retry:
                try:
                    shutil.copy(fname, "C:\\Users\\Asylum User\\Documents\\AEtesting\\copy.ibw")
                    # data = bw.load("C:\\Users\\Asylum User\\Documents\\AEtesting\\copy.ibw")
                    # wave = data['wave']['wData']
                    return load_ibw(fname)
                except FileNotFoundError:
                    print("File not found. Retrying {} times...".format(retries), end='\r')
                    retries += 1
                    time.sleep(wait)
                except PermissionError:
                    print("Permission denied. Retrying {} times...".format(retries), end='\r')
                    retries += 1
                    time.sleep(wait)
        else:
            return load_ibw(fname)
            # return bw.load(fname)['wave']['wData'].T
    else:
        download_file(connection=connection, file_path=fname, local_file_name='temp.ibw')
        time.sleep(wait)
        return load_ibw('temp.ibw')
        # return bw.load('temp.ibw')['wave']['wData'].T
    return 0

# Acquire a scan

In [ ]:
exp.execute('DownScan')

w = exp.load_ibw(folder=exp.folder)

exp.load_ibw_parameter(w)

plt.imshow(w.data[0], origin='lower')

In [ ]:
## atomai patches
coordinates = get_coord_grid(img, step = 1, return_dict=False)

# extract subimage for each point on a grid
window_size = 10
features_all, coords, _ = extract_subimages(img, coordinates, window_size)
features_all = features_all[:,:,:,0]
coords = np.array(coords, dtype=int)
indices_all = coords

print(coords.shape)
print(features_all.shape)

# see a patch : what atomai gave
k = 200

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(6, 3), dpi =100)
ax1.imshow(img)
ax1.scatter(coords[k, 1], coords[k, 0], marker='X', s=50, c='r')


ax2.imshow(features_all[k])

In [ ]:
coordinates = aoi.utils.get_coord_grid(img, step=1, return_dict=False)
features, targets, indices = aoi.utils.extract_patches_and_spectra(
    specim, img, coordinates=coordinates, window_size=window_size, avg_pool=1)

In [ ]:
norm_ = lambda x: (x - x.min()) / x.ptp()
features = norm_(features_all)

n, d1, d2 = features.shape
X = features.reshape(n, d1*d2)
X.shape

# use only 0.02% of grid data points as initial training points
(X_measured, X_unmeasured, indices_measured, indices_unmeasured) = train_test_split(
      X, indices_all, test_size=0.999, shuffle=True, random_state=5)

seed_points = len(X_measured)
seed_points

np.savez("output/seeds_PTO_20240401_3.npz", X_measured = X_measured, X_unmeasured = X_unmeasured, 
         indices_measured = indices_measured, indices_unmeasured = indices_unmeasured)



In [4]:
import os
f1 = r'C:\Users\Asylum User\Documents\AEtesting\ToIgor.arcmd'
    
f2 = r'C:\Users\Asylum User\Documents\AEtesting\SendToIgor.bat'

os.path.exists(f1)

False

In [5]:
if os.path.exists(f1):
    with open(f1, 'w') as fopen:
        fopen.write('')

In [6]:
dir_path = os.path.dirname(f1)

# Check if the directory exists, if not, create it
if not os.path.exists(dir_path):
    os.makedirs(dir_path)

# Create the file
with open(f1, 'w') as file:
    file.write('')

PermissionError: [WinError 5] Access is denied: 'C:\\Users\\Asylum User'